In [1]:
from __future__ import print_function, division
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D ,Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from PIL import Image
from PIL import ImageFilter
from tqdm import *
import keras.backend as K
import glob
import matplotlib.pyplot as plt

import sys

import numpy as np

/home/utkarsh/.conda/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
class WGAN():
    def __init__(self):
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=self.wasserstein_loss, 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss=self.wasserstein_loss, optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=( self.channels,self.img_rows, self.img_cols))
        img = self.generator(z)
        print(img.shape)
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss, 
            optimizer=optimizer,
            metrics=['accuracy'])
        self.files=[]
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):
        noise_shape = ( self.channels,self.img_rows, self.img_cols)
        model = Sequential()
        
        model.add(Conv2D(1024, kernel_size=3, strides=2, input_shape=noise_shape,data_format="channels_first", padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2DTranspose(32, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2DTranspose(32, kernel_size=2, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2DTranspose(16, kernel_size=2, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2DTranspose(8, kernel_size=2, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Conv2DTranspose(8, kernel_size=2, strides=1, padding="same"))
        model.add(Conv2DTranspose(8, kernel_size=2, strides=1, padding="same"))
        model.add(Conv2DTranspose(8, kernel_size=2, strides=1, padding="same"))
      
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = ( self.channels,self.img_rows, self.img_cols)
        
        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())

        model.summary()

        img = Input(shape=img_shape)
        features = model(img)
        valid = Dense(1, activation="linear")(features)

        return Model(img, valid)

    def train(self, epochs, batch_size=16, save_interval=50):

        # Load the dataset
        count = 1000
        for filename in tqdm(glob.iglob('data/celeba/origin/*.jpg', recursive=True)):
            img = Image.open(filename)
            self.files.append(np.array(img))
            img.close()
            count-=1
            if (count == 0):
                break
        self.files=np.array(self.files[:1000])

        # Rescale -1 to 1
        self.files = (self.files.astype(np.float32))
#         X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random half batch of images
                idx = np.random.randint(0, self.files.shape[0], half_batch)
                imgs = self.files[idx]
                noise = []
                for i in imgs:
                    print (i.shape)
                    im = Image.fromarray(i.astype('uint8'))
                    im = np.array(im.filter(ImageFilter.GaussianBlur(radius = 4)))
                    noise.append(im/((im.max())/255.0))
                noise = np.array(noise).transpose(0,3,1,2)
                imgs = np.array(imgs).transpose(0,3,1,2)
#                 noise = np.random.normal(0, 1, (half_batch, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict(noise)
            
                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, -np.ones((half_batch, 1)))
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.ones((half_batch, 1)))
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip discriminator weights
                for l in self.discriminator.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)


            # ---------------------
            #  Train Generator
            # ---------------------
            idx = np.random.randint(0, self.files.shape[0], half_batch)
            imgs = self.files[idx]
            noise = []
            for i in imgs:
                im = Image.fromarray(i.astype('uint8'))
                im = np.array(im.filter(ImageFilter.GaussianBlur(radius = 4)))
                noise.append(im/((im.max())/255.0))
            noise = np.array(noise).transpose(0,3,1,2)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, -np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        idx = np.random.randint(0, self.files.shape[0], half_batch)
        imgs = self.files[idx]
        noise = []
        for i in imgs:
            im = Image.fromarray(i.astype('uint8'))
            im = np.array(im.filter(ImageFilter.GaussianBlur(radius = 4)))
            noise.append(im/((im.max())/255.0))
        noise = np.array(noise).transpose(1, 2, 0)
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 1

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("wgan/images/mnist_%d.png" % epoch)
        plt.close()


In [8]:
wgan = WGAN()
wgan.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 2, 64, 16)         18448     
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 2, 64, 16)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2, 64, 16)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 1, 32, 32)         4640      
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 2, 33, 32)         0         
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 2, 33, 32)         0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 2, 33, 32)         0         
__________

ValueError: number of input channels does not match corresponding dimension of filter, 8 != 128

In [ ]:
import tensorflow, keras
print(tensorflow.__version__, keras.__version__)